# 远程数据库 

```{note}
    注:MDSplus数据库的访问，仅在等离子所内网有效。
```


在**SpDM**中，使用open_entry()建立统一的访问入口,支持远程MDSplus Server数据库的访问，及本地MDSplus的数据库中数据的访问。支持两个有用的功能：

（1）**不同数据源的自动集成**

SpDM中提供**open_entry**建立访问链接，**wall，pf，tf，magnetics**等被映射的数据均可以通过该链接入口访问。数据源来自于静态的XML文件，动态的MDS数据库中的不同tree：east,pcs_east,efit_east等

SpDM中数据的访问方式是按照IDS的树状结构逐层访问。

（2）**针对大型的数据库系统，“指标游走，懒惰执行”发挥重要作用**

SpDM的Entry支持指标游走，懒惰加载数据功能。这对大型的数据库系统非常重要。SpDM后台已经自动集成了不同数据源的数据，对应于不同的IDS条目中。所请求的条目或者某个条目中可能存储了大量的数据，如果请求的时候便立即从底层访问层后端读取所有数据，可能需要很长时间才能完成。通常情况下，用户可能只需要个别的数据子集，”懒惰加载”使得用户仅仅建立链接，只有在需要的时候才真正获取数据，这样会利于加速。

- **.child(path)**：将链接指针移动到指定的树节点，建立新的链接。返回是新的path。
- **.get(path)**：获得给定path节点的全部数据。返回时具体数值。（get=child+fetch）

## 针对数据库系统的URI表达式：
### 访问远程数据库系统
```

entry = open_entry("device+mdsplus://202.127.204.12?enable=efit_east&shot={shot_num}")

```
- device : 指定数据来源的装置
- mdsplus： 处理对象是MDS数据库
- 202.127.204.12 ：EAST mds数据库访问地址
- enable：同时可以获取efit_east数据
- shot：指定炮号


### 访问本地的MDSplus数据库
```
···
entry_local = open_entry(f"east+mdsplus://{DATA_PATH}/mdsplus/~t/?enable=efit_east&shot=70745")
```
- device : 指定数据来源的装置,如east
- mdsplus： 处理对象是MDS数据库
- {DATA_PATH}/mdsplus/~t/ 本地数据库的目录
- enable：同时可以获取efit_east数据
- shot：指定炮号

### 加载基本环境

In [2]:
### import基本环境
from spdm.core.entry import open_entry
from spdm.utils.logger import logger
import MDSplus
import os


In [3]:
### 指定mapping文件的路径
os.environ["SP_DATA_MAPPING_PATH"] = "/gpfs/fuyun/projects/fuyun/fytok/python/fytok/_mapping"

### 访问远程EAST MDSplus数据库

In [4]:
### 访问远程EAST MDS数据库中70754炮的数据，
shot_num = 70754
time_slice = 10
entry_reomte = open_entry("east+mdsplus://202.127.204.12?enable=efit_east&shot={shot_num}")

FileNotFoundError: Can not find mapping files for ('spdm/mapping/east', mdsplus://202.127.204.12?enable=efit_east,shot={shot_num}) MAPPING_PATH=[]!

In [ ]:
wall = entry_reomte.child("wall")

In [ ]:
wall.get()

### 访问远程本地MDSplus数据库

In [ ]:
# ### 访问本地MDSplus路径中中的70754炮的数据
shot_num = 70754
time_slice = 10
DATA_PATH = "/scratch/jupytertest/workspace_fytok/fytok_data"
entry_local = open_entry(f"east+mdsplus://{DATA_PATH}/mdsplus/~t/?shot=70745")

open_entry建立了指向数据库系统的链接entry_local，并且自动集成了已经映射的wall，pf，tf，magnetics等数据。

entry_local作为访问这些数据的入口。自动继成Entry的一切针对指针和数据的操作。

In [ ]:
### .child操作会将链接进一步指向wall
wall = entry_local.child("wall")
### 打印wall的类型，仍然是个entry
type(wall)
wall.get()

In [ ]:
### 支持按照连续字符串格式，继续移动链接到wall中的下一个子节点中,outline仍然是个entry
outline = entry_local.child("wall.description_2d[0].limiter.unit[0].outline")
### 获取outline的数据
print(outline.get())

#### SpDM中数据以字典形式在内存中直接交互

In [ ]:
[entry_local.get("wall").keys()]

#### SpDM中list的访问

In [ ]:
### 例如，b_field_pol_probe探针有多个，以list形式存在
type(entry_local.get("magnetics")["b_field_pol_probe"])


In [ ]:

### 每个探针下面是一个字典结构，获得其keys
entry_local.get("magnetics.b_field_pol_probe[0]").keys()


In [ ]:

entry_local.child("magnetics.b_field_pol_probe[0].position").fetch()

#### SpDM中数据按照树状结构层层访问

In [ ]:
### SpDM中数据按照层层路径进行访问:wall.description_2d[0].limiter.unit.outlin
### 获取outline的数据：
entry_local.get("wall.description_2d[0].limiter.unit.outline")